In [ ]:
import pandas as pd
from geopy.distance import geodesic
from itertools import combinations

# Update the file path to match your local directory
file_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"

# Load the dataset (modify sheet name if necessary)
df = pd.read_excel(file_path, sheet_name="Sheet1")  # Change sheet name if needed

# Ensure required columns are present
required_columns = {'ID', 'Latitude', 'Longitude', 'Class'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")

print("Columns found. Proceeding with filtering and distance calculations...")

# Filter only rows where Class is 'VI'
df_filtered = df[df['Class'] == 'VI']

# Create an edge list
edge_list = []

# Compute all-to-all distances for filtered rows only
for (id1, lat1, lon1), (id2, lat2, lon2) in combinations(df_filtered[['ID', 'Latitude', 'Longitude']].values, 2):
    distance_km = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    edge_list.append([id1, id2, distance_km, "Sink"])
    edge_list.append([id2, id1, distance_km, "Sink"])  # Add reverse direction

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "Type"])

# Save to CSV
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_node_edgelist_filtered.csv"
edge_df.to_csv(output_path, index=False)

print(f"Filtered edge list with distances saved as '{output_path}'.")


In [ ]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
from itertools import product

# File paths
sink_nodes_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Output_Geospatial_Analysis_Final_filled.xlsx"
candidate_nodes_path = r"C:\Users\himu1\Desktop\project\Project1\Databases_Sreekanta\Databases_Sreekanta\random_points1000.shp"
class_ii_shut_in_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_NETL_SONRIS_VI_Dry.shp"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_edgelist_filtered.csv"

# Load sink nodes from Excel
df_all = pd.read_excel(sink_nodes_path, sheet_name="Sheet1")

# Load candidate nodes from shapefiles
gdf_candidates_1 = gpd.read_file(candidate_nodes_path)
gdf_candidates_2 = gpd.read_file(class_ii_shut_in_path)

# Convert geometries to WGS84 and extract lat/lon
for gdf in [gdf_candidates_1, gdf_candidates_2]:
    gdf.to_crs(epsg=4326, inplace=True)
    gdf['Latitude'] = gdf.geometry.y
    gdf['Longitude'] = gdf.geometry.x

# Filter Class VI for From nodes
df_from = df_all[df_all['Class'] == 'VI'][['ID', 'Latitude', 'Longitude']]

# Filter candidates from all 3 sources based on class
valid_to_classes = {"SHUT-IN DRY HOLE -FUTURE UTILITY", "RANDOM EXPERT GUIDED"}
df_to = df_all[df_all['Class'].isin(valid_to_classes)][['ID', 'Latitude', 'Longitude']]
df_to = pd.concat([
    df_to,
    gdf_candidates_1[gdf_candidates_1['Class'].isin(valid_to_classes)][['ID', 'Latitude', 'Longitude']],
    gdf_candidates_2[gdf_candidates_2['Class'].isin(valid_to_classes)][['ID', 'Latitude', 'Longitude']]
], ignore_index=True)

# Sanity check on coordinate ranges
df_from = df_from[df_from['Latitude'].between(-90, 90) & df_from['Longitude'].between(-180, 180)]
df_to = df_to[df_to['Latitude'].between(-90, 90) & df_to['Longitude'].between(-180, 180)]

# Check if we have valid data
if df_from.empty or df_to.empty:
    print("No matching 'From' (Class VI) or 'To' (candidate) nodes found.")
    exit()

# Compute geodesic distances
edge_list = []
for (from_id, from_lat, from_lon), (to_id, to_lat, to_lon) in product(
    df_from.values, df_to.values
):
    distance_km = geodesic((from_lat, from_lon), (to_lat, to_lon)).kilometers
    edge_list.append([from_id, to_id, distance_km, "Candidate"])

# Save edge list
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "Type"])
edge_df.to_csv(output_path, index=False)

print(f"✅ Filtered edge list with distances saved as '{output_path}'.")


In [ ]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
from itertools import product

# 📂 File Paths
sink_nodes_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Output_Geospatial_Analysis_Final_filled.xlsx"
population_nodes_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_Pop_Cluster.shp"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_population_edgelist.csv"

# 📌 Load sink/candidate nodes
df_sinks = pd.read_excel(sink_nodes_path, sheet_name="Sheet1")

# 📌 Load census tract centroid data (FIPS, population)
gdf_population = gpd.read_file(population_nodes_path)

# Check geometry types before extracting coordinates
print("Geometry types in population dataset:", gdf_population.geom_type.unique())

# Convert to standard latitude/longitude (EPSG:4326)
gdf_population = gdf_population.to_crs(epsg=4326)

# Convert polygons to centroids if needed
if gdf_population.geom_type.iloc[0] != "Point":
    print("Converting polygons to centroids...")
    gdf_population["geometry"] = gdf_population.centroid

# Extract latitude and longitude
gdf_population['Latitude'] = gdf_population.geometry.y
gdf_population['Longitude'] = gdf_population.geometry.x

# Required columns check
required_columns_pop = {'FIPS', 'Latitude', 'Longitude', 'POP_SQMI', 'POPULATION'}
if not required_columns_pop.issubset(gdf_population.columns):
    raise ValueError(f"Missing columns in population dataset: {required_columns_pop - set(gdf_population.columns)}")

# Ensure latitude/longitude values are valid
df_sinks = df_sinks[(df_sinks['Latitude'].between(-90, 90)) & (df_sinks['Longitude'].between(-180, 180))]
gdf_population = gdf_population[(gdf_population['Latitude'].between(-90, 90)) & (gdf_population['Longitude'].between(-180, 180))]

# Define 30-mile radius filter function
def within_30_miles(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).miles <= 30

# Create edge list
edge_list = []

# Compute distances from each sink/candidate to census tract centroid
for (node_id, node_lat, node_lon), (fips, pop_lat, pop_lon, tract_densqmi, tot_pop) in product(
    df_sinks[['ID', 'Latitude', 'Longitude']].values,
    gdf_population[['FIPS', 'Latitude', 'Longitude', 'POP_SQMI', 'POPULATION']].values
):
    if within_30_miles(node_lat, node_lon, pop_lat, pop_lon):
        distance_km = geodesic((node_lat, node_lon), (pop_lat, pop_lon)).kilometers
        edge_list.append([node_id, fips, distance_km, tract_densqmi, tot_pop, "PopulationCluster"])

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "Tract_Densqmi", "TotPop", "Type"])

# Save to CSV
edge_df.to_csv(output_path, index=False)

print(f"Filtered edge list with census tract distances saved as '{output_path}'.")


In [ ]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
from itertools import product

# 📂 File Paths
ejs_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Output_Geospatial_Analysis_Final_filled.xlsx"
source_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_CO2_Source.shp"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_source_edgelist.csv"

# 📌 Load Sink Candidate Nodes from Excel
df_ejs = pd.read_excel(ejs_data_path, sheet_name="Sheet1", engine="openpyxl")

# 📌 Load CO2 Source Nodes
gdf_source = gpd.read_file(source_data_path)

# Ensure 'GHG_QUAN_5' exists
if 'GHG_QUAN_5' not in gdf_source.columns:
    raise ValueError("⚠ 'GHG_QUAN_5' column missing in source dataset!")

# Rename columns
df_ejs.rename(columns={'ID': 'SinkCandidateID'}, inplace=True)
gdf_source.rename(columns={'GHGRP_ID': 'SourceID', 'GHG_QUAN_5': 'TonsCO2e'}, inplace=True)

# Ensure Geometry column exists and extract Latitude/Longitude
gdf_source = gdf_source.to_crs(epsg=4326)
gdf_source['Latitude'] = gdf_source.geometry.y
gdf_source['Longitude'] = gdf_source.geometry.x

# Drop missing lat/lon values
df_ejs = df_ejs.dropna(subset=['Latitude', 'Longitude'])
gdf_source = gdf_source.dropna(subset=['Latitude', 'Longitude', 'TonsCO2e'])

# Ensure lat/lon values are valid
df_ejs['Latitude'] = df_ejs['Latitude'].clip(lower=-90, upper=90)
df_ejs['Longitude'] = df_ejs['Longitude'].clip(lower=-180, upper=180)

# Create an edge list
edge_list = []

# Compute distances
for (sink_id, lat1, lon1), (source_id, lat2, lon2, tons_co2e) in product(
    df_ejs[['SinkCandidateID', 'Latitude', 'Longitude']].values,
    gdf_source[['SourceID', 'Latitude', 'Longitude', 'TonsCO2e']].values
):
    distance_km = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    if distance_km <= 160.93:  # 100 miles in km
        edge_list.append([sink_id, source_id, distance_km, tons_co2e, "Source"])

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "TonsCO2e", "Type"])

# Save to CSV
edge_df.to_csv(output_path, index=False)

In [1]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning

# Define file paths (update these with your correct paths)
sink_candidate_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Output_Geospatial_Analysis_Final_filled.xlsx"
pipeline_shapefile_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_CO2Pipe.shp"

# Load the sink/candidate nodes data
try:
    print("Loading sink/candidate data...")
    sink_candidate_df = pd.read_excel(sink_candidate_path)
except Exception as e:
    print(f"Error loading excel file: {e}")
    sink_candidate_df = pd.read_csv(sink_candidate_path)

# Load the pipeline data as GeoDataFrame
print("Loading pipeline shapefile...")
pipeline_gdf = gpd.read_file(pipeline_shapefile_path)

# Print column information for debugging
print("Sink candidate columns:", sink_candidate_df.columns.tolist())
print("Pipeline columns:", pipeline_gdf.columns.tolist())

# Ensure required columns exist in pipeline data
required_pipeline_columns = {'ID', 'DIAMETER', 'LENGTH_MIL', 'geometry'}
missing_pipeline_columns = required_pipeline_columns - set(pipeline_gdf.columns)

if missing_pipeline_columns:
    raise ValueError(f"⚠ Missing columns in pipeline dataset: {missing_pipeline_columns}")

# Ensure required columns exist in sink/candidate data
if 'Latitude' not in sink_candidate_df.columns or 'Longitude' not in sink_candidate_df.columns:
    raise ValueError(f"⚠ Missing 'Latitude' or 'Longitude' columns in sink candidate dataset!")

# Process sink/candidate data
sink_candidate_df = sink_candidate_df.dropna(subset=['Latitude', 'Longitude'])
geometry = [Point(xy) for xy in zip(sink_candidate_df['Longitude'], sink_candidate_df['Latitude'])]
sink_candidate_gdf = gpd.GeoDataFrame(sink_candidate_df, geometry=geometry, crs="EPSG:4326")

# Ensure 'ID' column exists in sink candidate data
if 'ID' in sink_candidate_gdf.columns:
    sink_candidate_gdf['From'] = sink_candidate_gdf['ID']
else:
    sink_candidate_gdf['From'] = [f"Sink_{i}" for i in range(len(sink_candidate_gdf))]

# Ensure CRS consistency
if sink_candidate_gdf.crs != pipeline_gdf.crs:
    sink_candidate_gdf = sink_candidate_gdf.to_crs(pipeline_gdf.crs)

# Function to calculate distance in kilometers to a linestring (pipeline)
def calculate_distance_to_line_km(point, line):
    if not isinstance(point, gpd.GeoSeries):
        point = gpd.GeoSeries([point], crs=sink_candidate_gdf.crs)
    if not isinstance(line, gpd.GeoSeries):
        line = gpd.GeoSeries([line], crs=pipeline_gdf.crs)

    if point.crs.is_geographic:
        point = point.to_crs(epsg=3857)  # Web Mercator projection
        line = line.to_crs(epsg=3857)

    return point.distance(line).values[0] / 1000  # Convert meters to km

# Create empty lists to store edge data
from_nodes = []
to_nodes = []
distances_km = []
pipe_diameters = []
pipe_lengths = []

# For each sink/candidate node, find distance to all pipeline nodes
print("Calculating distances between sink nodes and pipeline nodes...")
total_count = len(sink_candidate_gdf) * len(pipeline_gdf)
count = 0
progress_interval = max(1, total_count // 20)  # Show progress at 5% intervals

for idx, sink in sink_candidate_gdf.iterrows():
    sink_id = sink['From']
    sink_point = sink.geometry
    
    for pipe_idx, pipeline in pipeline_gdf.iterrows():
        count += 1
        if count % progress_interval == 0:
            print(f"Progress: {count/total_count*100:.1f}% ({count}/{total_count})")
            
        pipe_id = pipeline['ID']
        pipe_geometry = pipeline.geometry
        pipe_diameter = pipeline['DIAMETER']
        pipe_length = pipeline['LENGTH_MIL']

        try:
            # Calculate distance to pipeline
            dist_km = calculate_distance_to_line_km(sink_point, pipe_geometry)
            
            # Store the data
            from_nodes.append(sink_id)
            to_nodes.append(pipe_id)
            distances_km.append(dist_km)
            pipe_diameters.append(pipe_diameter)
            pipe_lengths.append(pipe_length)
                
        except Exception as e:
            print(f"Error calculating distance between {sink_id} and {pipe_id}: {e}")
            continue

# Create edge list dataframe
print("Creating edge list...")
edge_data = {
    'From': from_nodes,
    'To': to_nodes,
    'Relational process description': 'Sink/candidate node to pipeline node',
    'Edge Attribute': 'Distance, Diameter, Length',
    'Column Names': 'Dist_Km, Pipe_Diam, Pipe_Length',
    'Type': 'Pipeline',
    'Dist_Km': distances_km,
    'Pipe_Diam': pipe_diameters,
    'Pipe_Length': pipe_lengths
}

edge_list_df = pd.DataFrame(edge_data)

# Save the edge list to a CSV file
output_path = r"C:\Users\himu1\Desktop\project\Project2\sink_pipeline_edge_list.csv"
edge_list_df.to_csv(output_path, index=False)

# Display completion message
print(f"🎯 Analysis complete. {len(edge_list_df)} connections found between sink nodes and pipeline nodes.")
print(f"📁 Edge list saved to: {output_path}")


Loading sink/candidate data...
Loading pipeline shapefile...
Sink candidate columns: ['ID', 'Latitude', 'Longitude', 'Class', 'Otherwell_density', 'Refwell_density', 'Refwell_DISTm', 'CrA_Presence', 'CrA_DISTm', 'PAD_Presence', 'PAD_DISTm', 'PAD_CAT_1', 'PAD_CAT_2', 'GW_DISTm', 'GW_AquiferDes', 'GW_DEPTHft', 'GW_AquiferName', 'WL_Presence', 'WL_DISTm', 'WL_Type', 'EJS_TotPop', 'EJS_TractID', 'EJS_TractLandm', 'EJS_TractWaterm', 'EJS_OZONE', 'EJS_DIESEL', 'EJS_ToxicAIR', 'EJS_TRAFFIC', 'EJS_LEADPaint', 'EJS_Superfund', 'EJS_RMP', 'EJS_HAZWaste', 'EJS_UnderTanks', 'EJS_WasteWater', 'EJS_NO2', 'EJS_DrinkWater', 'NHD_Presence', 'NHD_DISTm', 'NHD_WBType_1', 'NHD_WBType_2', 'AUS_Presence', 'AUS_DISTm', 'AUS_Name_1', 'PAR_Presence', 'PAR_OWNType', 'PAR_SubOWNType', 'PAR_Zone', 'PAR_ZoneDesc', 'PAR_STRClass', 'PAR_STRType', 'PAR_Areaft', 'COAST_Presence', 'Dep_Frio_Delta_Presence', 'Dep_Frio_Areakm2', 'Dep_Frio_LayerName', 'Dep_LowerMio1_Delta_Presence', 'Dep_LowerMio1_Areakm2', 'Dep_LowerMio1

In [2]:
# Import necessary libraries
import pandas as pd
import os
import glob
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning

# Define file paths
base_path = r"C:\Users\himu1\Desktop\project\Project2\Edge list"
output_file_path = os.path.join(base_path, "merged_edge_list.csv")

# Define the order of files to process
file_types = ["Sink", "Candidate", "Population", "Source", "Pipeline"]

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Process each file type in the specified order
for file_type in file_types:
    # Find files that match the current type
    pattern = os.path.join(base_path, f"*{file_type}*.csv")
    matching_files = glob.glob(pattern)
    
    if not matching_files:
        print(f"Warning: No files found matching pattern '{pattern}'")
        continue
    
    # Use the first matching file
    file_path = matching_files[0]
    print(f"Processing file: {os.path.basename(file_path)}")
    
    try:
        # Read the current file
        current_df = pd.read_csv(file_path)
        
        # Display basic information about the file
        print(f"  Columns: {current_df.columns.tolist()}")
        print(f"  Rows: {len(current_df)}")
        
        # Keep track of any additional columns in this file
        additional_columns = [col for col in current_df.columns if col not in ["From", "To", "Dist_Km", "Type"]]
        
        if additional_columns:
            print(f"  Additional columns found: {additional_columns}")
        
        # If this is the first file, use it to initialize the merged DataFrame
        if merged_df.empty:
            merged_df = current_df.copy()
        else:
            # Get all columns from both DataFrames
            all_columns = set(merged_df.columns) | set(current_df.columns)
            
            # For each DataFrame, add missing columns with NA values
            for col in all_columns:
                if col not in merged_df.columns:
                    merged_df[col] = pd.NA
                if col not in current_df.columns:
                    current_df[col] = pd.NA
            
            # Append the current DataFrame to the merged one
            merged_df = pd.concat([merged_df, current_df], ignore_index=True)
        
    except Exception as e:
        print(f"  Error processing file {file_path}: {e}")

# Display information about the merged DataFrame
print("\nMerged DataFrame:")
print(f"  Total rows: {len(merged_df)}")
print(f"  All columns: {merged_df.columns.tolist()}")

# Save the merged DataFrame to a CSV file
merged_df.to_csv(output_file_path, index=False)
print(f"\nMerged edge list saved to: {output_file_path}")

# Display a sample of the merged data
print("\nSample of merged data:")
print(merged_df.head())

# Optional: Display counts by Type for verification
type_counts = merged_df['Type'].value_counts()
print("\nRows by Type:")
for type_name, count in type_counts.items():
    print(f"  {type_name}: {count}")

Processing file: Sink.csv
  Columns: ['From', 'To', 'Dist_Km', 'Type']
  Rows: 380
Processing file: Candidate.csv
  Columns: ['From', 'To', 'Dist_Km', 'Type']
  Rows: 63200
Processing file: Population.csv
  Columns: ['From', 'To', 'Dist_Km', 'Type', 'Tract_Densqmi', 'TotPop']
  Rows: 109609
  Additional columns found: ['Tract_Densqmi', 'TotPop']
Processing file: Source.csv
  Columns: ['From', 'To', 'Dist_Km', 'Type', 'TonsCO2']
  Rows: 266420
  Additional columns found: ['TonsCO2']
Processing file: Pipeline.csv
  Columns: ['From', 'To', 'Dist_Km', 'Type', 'Pipe_Diam', 'Pipe_Length']
  Rows: 84188
  Additional columns found: ['Pipe_Diam', 'Pipe_Length']

Merged DataFrame:
  Total rows: 523797
  All columns: ['From', 'To', 'Dist_Km', 'Type', 'Tract_Densqmi', 'TotPop', 'TonsCO2', 'Pipe_Diam', 'Pipe_Length']

Merged edge list saved to: C:\Users\himu1\Desktop\project\Project2\Edge list\merged_edge_list.csv

Sample of merged data:
     From      To     Dist_Km  Type  Tract_Densqmi TotPop Ton